In [1]:
import pandas as pd
import re
import emoji
import numpy as np

In [2]:
import nltk

In [3]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
df=pd.read_csv("/root/celebrity_profiling_training-dataset_subset_5_tweets.csv",engine="python",error_bad_lines=False,usecols=['id','text','occupation','birthyear','gender'])

In [5]:
df.head()

,birthyear,gender,id,occupation,text
0,1996.0,male,1.0,sports,DOTA 2 Livestream India • Pro Dota 2 India Gam...
1,1996.0,male,1.0,sports,DOTA 2 Livestream India • Immortal Gameplay | ...
2,1996.0,male,1.0,sports,DOTA 2 Livestream India • Immortal Gameplay | ...
3,1996.0,male,1.0,sports,DOTA 2 Livestream India • The Outlanders | !he...
4,1996.0,male,1.0,sports,DOTA 2 Livestream India • The Outlanders | !he...


In [6]:
df.shape

(664118, 5)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 664118 entries, 0 to 664117
Data columns (total 5 columns):
birthyear     664035 non-null float64
gender        664035 non-null object
id            664035 non-null float64
occupation    664035 non-null object
text          664035 non-null object
dtypes: float64(2), object(3)
memory usage: 25.3+ MB


In [8]:
set(df['occupation'])

{None, 'creator', 'performer', 'politics', 'sports'}

In [9]:
df['occupation'] = df['occupation'].replace('creator', 0)
df['occupation'] = df['occupation'].replace('performer', 1)
df['occupation'] = df['occupation'].replace('politics', 2)
df['occupation'] = df['occupation'].replace('sports', 3)

In [10]:
df.head()

,birthyear,gender,id,occupation,text
0,1996.0,male,1.0,3.0,DOTA 2 Livestream India • Pro Dota 2 India Gam...
1,1996.0,male,1.0,3.0,DOTA 2 Livestream India • Immortal Gameplay | ...
2,1996.0,male,1.0,3.0,DOTA 2 Livestream India • Immortal Gameplay | ...
3,1996.0,male,1.0,3.0,DOTA 2 Livestream India • The Outlanders | !he...
4,1996.0,male,1.0,3.0,DOTA 2 Livestream India • The Outlanders | !he...


In [11]:
df['gender'] = df['gender'].replace('female', 0)
df['gender'] = df['gender'].replace('male', 1)

In [12]:
df.head()

,birthyear,gender,id,occupation,text
0,1996.0,1.0,1.0,3.0,DOTA 2 Livestream India • Pro Dota 2 India Gam...
1,1996.0,1.0,1.0,3.0,DOTA 2 Livestream India • Immortal Gameplay | ...
2,1996.0,1.0,1.0,3.0,DOTA 2 Livestream India • Immortal Gameplay | ...
3,1996.0,1.0,1.0,3.0,DOTA 2 Livestream India • The Outlanders | !he...
4,1996.0,1.0,1.0,3.0,DOTA 2 Livestream India • The Outlanders | !he...


df['birthyear'] = np.where(df['birthyear'].between(1995,1999), 0, df['birthyear'])
df['birthyear'] = np.where(df['birthyear'].between(1988,1994), 1, df['birthyear'])
df['birthyear'] = np.where(df['birthyear'].between(1979,1987), 2, df['birthyear'])
df['birthyear'] = np.where(df['birthyear'].between(1968,1978), 3, df['birthyear'])
df['birthyear'] = np.where(df['birthyear'].between(1955,1967), 4, df['birthyear'])
df['birthyear'] = np.where(df['birthyear'].between(1940,1954), 5, df['birthyear'])

In [13]:
df['birthyear'] = np.where(df['birthyear'].between(1990,1999), 0, df['birthyear'])
df['birthyear'] = np.where(df['birthyear'].between(1981,1989), 1, df['birthyear'])
df['birthyear'] = np.where(df['birthyear'].between(1970,1980), 2, df['birthyear'])
df['birthyear'] = np.where(df['birthyear'].between(1956,1969), 3, df['birthyear'])
df['birthyear'] = np.where(df['birthyear'].between(1940,1955), 4, df['birthyear'])

In [14]:
df.head()

,birthyear,gender,id,occupation,text
0,0.0,1.0,1.0,3.0,DOTA 2 Livestream India • Pro Dota 2 India Gam...
1,0.0,1.0,1.0,3.0,DOTA 2 Livestream India • Immortal Gameplay | ...
2,0.0,1.0,1.0,3.0,DOTA 2 Livestream India • Immortal Gameplay | ...
3,0.0,1.0,1.0,3.0,DOTA 2 Livestream India • The Outlanders | !he...
4,0.0,1.0,1.0,3.0,DOTA 2 Livestream India • The Outlanders | !he...


In [15]:
sentences=list(df.text)

In [16]:
sentences[0]

'DOTA 2 Livestream India • Pro Dota 2 India Gameplay | !health https://t.co/CuXGznXRCi'

Tweet Cleaner

I combined various text cleaning techniques. The original source of each technique is credited alongside the relevant block of code. Features include:

Expands contractions (can't = cannot)
Expands tweet slang (OMG = Oh My God)
Removes emojis
Removes HTML tags
Removes URLs
Replaces repeated punctuation with a single character (Wow!!!!! = Wow!)
Removes special characters

In [17]:
# https://stackoverflow.com/a/34682849
from bs4 import BeautifulSoup
import unicodedata
from nltk.tokenize import word_tokenize

def untokenize(words):
    """Untokenizing a text undoes the tokenizing operation, restoring
    punctuation and spaces to the places that people expect them to be.
    Ideally, `untokenize(tokenize(text))` should be identical to `text`,
    except for line breaks.
    """
    text = ' '.join(words)
    step1 = text.replace("`` ", '"').replace(" ''", '"').replace('. . .', '...')
    step2 = step1.replace(" ( ", " (").replace(" ) ", ") ")
    step3 = re.sub(r' ([.,:;?!%]+)([ \'"`])', r"\1\2", step2)
    step4 = re.sub(r' ([.,:;?!%]+)$', r"\1", step3)
    step5 = step4.replace(" '", "'").replace(" n't", "n't").replace(
        "can not", "cannot")
    step6 = step5.replace(" ` ", " '")
    return step6.strip()


# https://stackoverflow.com/a/47091490
def decontracted(phrase):
    """Convert contractions like "can't" into "can not"
    """
    # specific
    phrase = re.sub(r"won\'t", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    #phrase = re.sub(r"n't", " not", phrase) # resulted in "ca not" when sentence started with "can't"
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase


# https://github.com/rishabhverma17/sms_slang_translator/blob/master/slang.txt
slang_abbrev_dict = {
    'AFAIK': 'As Far As I Know',
    'AFK': 'Away From Keyboard',
    'ASAP': 'As Soon As Possible',
    'ATK': 'At The Keyboard',
    'ATM': 'At The Moment',
    'A3': 'Anytime, Anywhere, Anyplace',
    'BAK': 'Back At Keyboard',
    'BBL': 'Be Back Later',
    'BBS': 'Be Back Soon',
    'BFN': 'Bye For Now',
    'B4N': 'Bye For Now',
    'BRB': 'Be Right Back',
    'BRT': 'Be Right There',
    'BTW': 'By The Way',
    'B4': 'Before',
    'B4N': 'Bye For Now',
    'CU': 'See You',
    'CUL8R': 'See You Later',
    'CYA': 'See You',
    'FAQ': 'Frequently Asked Questions',
    'FC': 'Fingers Crossed',
    'FWIW': 'For What It\'s Worth',
    'FYI': 'For Your Information',
    'GAL': 'Get A Life',
    'GG': 'Good Game',
    'GN': 'Good Night',
    'GMTA': 'Great Minds Think Alike',
    'GR8': 'Great!',
    'G9': 'Genius',
    'IC': 'I See',
    'ICQ': 'I Seek you',
    'ILU': 'I Love You',
    'IMHO': 'In My Humble Opinion',
    'IMO': 'In My Opinion',
    'IOW': 'In Other Words',
    'IRL': 'In Real Life',
    'KISS': 'Keep It Simple, Stupid',
    'LDR': 'Long Distance Relationship',
    'LMAO': 'Laugh My Ass Off',
    'LOL': 'Laughing Out Loud',
    'LTNS': 'Long Time No See',
    'L8R': 'Later',
    'MTE': 'My Thoughts Exactly',
    'M8': 'Mate',
    'NRN': 'No Reply Necessary',
    'OIC': 'Oh I See',
    'OMG': 'Oh My God',
    'PITA': 'Pain In The Ass',
    'PRT': 'Party',
    'PRW': 'Parents Are Watching',
    'QPSA?': 'Que Pasa?',
    'ROFL': 'Rolling On The Floor Laughing',
    'ROFLOL': 'Rolling On The Floor Laughing Out Loud',
    'ROTFLMAO': 'Rolling On The Floor Laughing My Ass Off',
    'SK8': 'Skate',
    'STATS': 'Your sex and age',
    'ASL': 'Age, Sex, Location',
    'THX': 'Thank You',
    'TTFN': 'Ta-Ta For Now!',
    'TTYL': 'Talk To You Later',
    'U': 'You',
    'U2': 'You Too',
    'U4E': 'Yours For Ever',
    'WB': 'Welcome Back',
    'WTF': 'What The Fuck',
    'WTG': 'Way To Go!',
    'WUF': 'Where Are You From?',
    'W8': 'Wait',
    '7K': 'Sick:-D Laugher'
}


def unslang(text):
    """Converts text like "OMG" into "Oh my God"
    """
    if text.upper() in slang_abbrev_dict.keys():
        return slang_abbrev_dict[text.upper()]
    else:
        return text



# Reference : https://gist.github.com/slowkow/7a7f61f495e3dbb7e3d767f97bd7304b
def replace_emoji_to_text(text):
    return emoji.demojize(text)




def replace_urls(text):
    text=re.sub(r"http\S+", 'URL', text)
    text=re.sub(r"www\S+", 'URL', text)
    text=re.sub(r"pic.twitter.com\S+", 'URL', text)
    text=re.sub(r'https.*[^ ]', 'URL',text)
    
    return text

def clean(reg_exp, text):
    text = re.sub(reg_exp, " ", text)

    # replace multiple spaces with one.
    text = re.sub('\s{2,}', ' ', text)

    return text

def remove_accented_chars(text):
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return text

def strip_html_tags(text):
    soup = BeautifulSoup(text, "html.parser")
    stripped_text = soup.get_text()
    return stripped_text

def clean_all(t):
    
    # first do bulk cleanup on tokens that don't depend on word tokenization

    # remove xml tags
    t = clean(r"<[^>]+>", str(t))
    t = clean(r"&lt;", str(t))
    t = clean(r"&gt;", str(t))

    # remove URLs
    t = replace_urls(t)

    
    
    # Removing HTML tags
    t=strip_html_tags(t)
    
    #remove_accented_chars
    t=remove_accented_chars(t)

    # https://stackoverflow.com/a/35041925
    # replace multiple punctuation with single. Ex: !?!?!? would become ?
    t = clean(r'[\?\.\!]+(?=[\?\.\!])', t)

    t = replace_emoji_to_text(t)

    # expand common contractions like "I'm" "he'll"
    t = decontracted(t)

    # now remove/expand bad patterns per word
    words = word_tokenize(t)

    clean_words = []

    for w in words:
        # normalize punctuation
        w = re.sub(r'&', 'and', w)

        # expand slang like OMG = Oh my God
        w = unslang(w)


        clean_words.append(w)

    # join the words back into a full string
    t = untokenize(clean_words)


    # finally, remove any non ascii and special characters that made it through
    t = clean(r"[^A-Za-z0-9\.\'!\?,\$]", t)

    return t

In [18]:
text_re = re.compile("[^a-zA-Z\s]")
url_re = re.compile("http(s)*://[\w]+\.(\w|/)*(\s|$)")
hashtag_re = re.compile("[\W]#[\w]*[\W]")
mention_re = re.compile("(^|[\W\s])@[\w]*[\W\s]")
not_ascii_re = re.compile("([^\x00-\x7F]+)")
time_re = re.compile("(^|\D)[\d]+:[\d]+")
numbers_re = re.compile("(^|\D)[\d]+[.'\d]*\D")
space_collapse_re = re.compile("[\s]+")

In [19]:
sentences = [re.sub(mention_re, " <USER> ", str(sentence)) for sentence in sentences]
sentences = [re.sub(time_re, " <TIME> ", str(sentence)) for sentence in sentences]
sentences = [re.sub(numbers_re, " <NUMBER> ", str(sentence)) for sentence in sentences]
sentences = [re.sub(not_ascii_re, "", str(sentence)) for sentence in sentences]
sentences = [re.sub(space_collapse_re, " ", str(sentence)) for sentence in sentences]
sentences = [str(sentence).replace("\n", "") for sentence in sentences]

sentences = [clean_all(str(sentence)) for sentence in sentences]
sentences = [re.sub(r'#([^ ]*)', r'\1', str(sentence)) for sentence in sentences]
sentences = [re.sub(r'https.*[^ ]', 'URL', str(sentence)) for sentence in sentences]
sentences = [emoji.demojize(str(sentence)) for sentence in sentences]
sentences = [re.sub(r'(:.*?:)', r' \1 ', str(sentence)) for sentence in sentences]
sentences = [re.sub(' +', ' ', str(sentence)) for sentence in sentences]


/opt/conda/lib/python3.6/site-packages/bs4/__init__.py:219: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
/opt/conda/lib/python3.6/site-packages/bs4/__init__.py:219: UserWarning: "b'..'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)


In [20]:
sentences = [str(sentence).lower() for sentence in sentences]

In [21]:
sentences[0]

'dota livestream india pro dota india gameplay ! health url'

In [22]:
birthyear=df['birthyear']
gender=df['gender']
occupation=df['occupation']

In [23]:
sentences0=[]
gender0=[]
birthyear0=[]
occupation0=[]

for i in range(len(sentences)):
    if(len(str(sentences[i]))>30):
        sentences0.append(str(sentences[i]))
        birthyear0.append(int(birthyear[i]))
        gender0.append(int(gender[i]))
        occupation0.append(int(occupation[i]))


In [24]:
d = {'text':sentences0,'gender': gender0, 'occupation': occupation0,'birthyear': birthyear0}
df_after_work = pd.DataFrame(data=d)

In [25]:
df_after_work=df_after_work.sample(frac=1).reset_index(drop=True)

In [26]:
df_after_work=df_after_work.dropna()

In [27]:
df_after_work.head()

,birthyear,gender,occupation,text
0,0,1,1,"i am in new zealand, an i must say there are a..."
1,0,0,0,rt so a one piece highschool anime is coming o...
2,0,0,0,rt kobe bryant the conductor comercial da nike...
3,3,1,1,rt this is an embarrassing night for america. ...
4,1,1,3,legoncitiesfc head coach with the accra sports...


In [28]:
df_after_work.tail()

,birthyear,gender,occupation,text
513148,2,1,1,get this app to stream free music with no ads ...
513149,0,0,1,rt im one clothing purchase away from being ha...
513150,1,1,1,rt your brain automatically translates what th...
513151,2,1,0,rt too many christians are confused about how ...
513152,2,1,1,rt liverpool miss out on winning the premier l...


In [29]:
df_after_work.shape

(513153, 4)

In [30]:
df_after_work.to_csv('celebrity_profiling_training_after_preprocessing_5_tweets_2.csv', index=False)

In [ ]:
#engine="python",error_bad_lines=False,
#usecols=['id','text','occupation','birthyear','gender']